In [7]:
#以下の手順で必要なライブラリをインストールします。
#
#①まず、chromeのバージョンを調べます。
#　chromeの右上の設定から左一覧の一番下のchromeについて で開いた画面にバージョンが表示されます。
#  2025/5/1現在　136.0.7103.49
#
#②次に、https://pypi.org/project/chromedriver-binary/#history　を開いて、
#　chromedriver-binaryのバージョンの中で、chromeのバージョンを越えない一番最近のバージョン選びます。
#  2025/5/1現在 136.0.7103.49
#
#③Anaconda Prompt（スタートからAnaconda3(64bit) -> Anaconda Prompt　で）を開いて以下のコマンドを実行します。
#
#"pip install selenium"
#"pip install chromedriver-binary==136.0.7103.49"  (==の後に、上記で選んだバージョンを指定します。）
#
#
#まずは、試しに日経プレスリリースから指定したキーワードで検索した結果を確認する。
#指定するキーワードは、"パナソニック AND 発表日：2024"
#
import urllib.parse
from urllib.request import urlopen
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import chromedriver_binary
# 日経のプレスリリースを確認する
# まずは、日経のプレスリリースのページをブラウザで確認
# https://www.nikkei.com/pressrelease/
#　日経のプレスリリースを検索
# ブラウザーを起動
options = Options()
# 必要ならproxyサーバを設定する
#PROXY = '127.0.0.1:8080'
#options.add_argument('--proxy-server=http://%s' % PROXY)
# ブラウザはヘッドレス（表示しない）とする
options.add_argument('--headless')
# Chromeのドライバを得る
browser = webdriver.Chrome(options=options)

# 暗黙的な待機を最大3秒行う(サーバーの負担軽減)
browser.implicitly_wait(3)

# 検索キーワードをURLエンコードする
keywords = urllib.parse.quote('パナソニック　AND 発表日：2025')
url = 'https://www.nikkei.com/pressrelease/?searchKeyword='+ keywords + '&au=0'

# URLを読み込む
browser.get(url)
# htmlを取得
search_results_html = browser.page_source
print(search_results_html)

<html lang="ja" data-service-name="k2-stream" data-is-webview="false"><head><meta charset="utf-8"><meta name="viewport" content="width=device-width, initial-scale=1.0, minimum-scale=1"><link rel="preload" as="image" href="/.resources/k-components/icon/prime-veritas.rev-fdd0e06.svg"><link rel="preload" as="image" href="/.resources/k-components/icon/prime-digital-governance.rev-0b4eacf.svg"><link rel="preload" as="image" href="/.resources/k-components/icon/nikkei-financial.rev-1034954.svg"><link rel="preload" as="image" href="/.resources/k-components/icon/prime-ft-the-world.rev-e0334d4.svg"><link rel="preload" as="image" href="/.resources/k-components/icon/prime-gx.rev-734d3be.svg"><link rel="preload" as="image" href="/.resources/k-components/icon/prime-minutes.rev-e4faa83.svg"><link rel="preload" as="image" href="/.resources/k-components/icon/prime-mobility.rev-c395a1c.svg"><link rel="preload" as="image" href="/.resources/k-components/icon/prime-tech-foresight.rev-d36d231.svg"><link rel

In [9]:
# 記事本体の確認
url = "https://www.nikkei.com/article/DGXZRSP690485_Q5A430C2000000/"
# URLを読み込む
browser.get(url)
# htmlを取得
search_results_html = browser.page_source
print(search_results_html)

<html lang="ja" data-service-name="kite-article" data-is-webview="false"><head><meta charset="utf-8"><meta name="viewport" content="width=device-width, initial-scale=1.0, minimum-scale=1"><link rel="preload" as="image" href="/.resources/k-components/icon/prime-veritas.rev-fdd0e06.svg"><link rel="preload" as="image" href="/.resources/k-components/icon/prime-digital-governance.rev-0b4eacf.svg"><link rel="preload" as="image" href="/.resources/k-components/icon/nikkei-financial.rev-1034954.svg"><link rel="preload" as="image" href="/.resources/k-components/icon/prime-ft-the-world.rev-e0334d4.svg"><link rel="preload" as="image" href="/.resources/k-components/icon/prime-gx.rev-734d3be.svg"><link rel="preload" as="image" href="/.resources/k-components/icon/prime-minutes.rev-e4faa83.svg"><link rel="preload" as="image" href="/.resources/k-components/icon/prime-mobility.rev-c395a1c.svg"><link rel="preload" as="image" href="/.resources/k-components/icon/prime-tech-foresight.rev-d36d231.svg"><link 

In [11]:
# プレスリリースのスクレイピング
import sys
import urllib.parse
import re
from html import unescape
import datetime
import os

from urllib.request import urlopen
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import chromedriver_binary
from bs4 import BeautifulSoup

def get_article_to_text(search_results, file):
  old_url=''
  for article_local_url in re.findall(r'<a href="(.*?)"', search_results, re.DOTALL):
    if article_local_url[0:6]=="/press":
        continue
    if article_local_url[0:5]=="https":
        continue
    if article_local_url==old_url:
        old_url=''
        continue
    old_url=article_local_url
    article_url = 'https://www.nikkei.com'+ article_local_url
    print(article_url)
    # 暗黙的な待機を最大3秒行う(サーバーの負担軽減)
    browser.implicitly_wait(3)
    # URLを読み込む
    browser.get(article_url)
    # htmlを取得
    article_html = browser.page_source
    #print(article_html)
    header = re.search(r'<h1 class="title_t1xgcrem">(.*?)</h1>',article_html,re.DOTALL).group(1)
    date = re.search(r'<time datetime=(.*?)000Z">(.*?)</time>',article_html,re.DOTALL).group(2)
    print(header,date)
    # 記事本体からテキストのみを取り出し、テキストファイルに書き出す
    a = re.search(r'発表日:(.*?)すべての記事が読み放題',article_html,re.DOTALL).group(1)
    article = re.search(r'<p class="paragraph_p18mfke4">(.*?)アプリで開く', a, re.DOTALL).group(1)
    article = re.sub('<.*?>', '', article)  # 不要なタグをすべて取り除く
    try:
      file.write(article)
      file.write('\n')
    except:
      print(article)
  return

# 企業名　と　発表年　で日経のプレスリリースを検索するURLを作成
# 検索キーワードをURLエンコードする
keywords = urllib.parse.quote('パナソニック AND 発表日：2025')
url = 'https://www.nikkei.com/pressrelease/?searchKeyword='+ keywords + '&au=0'

#　日経のプレスリリースを検索
# ブラウザーを起動
options = Options()
# 必要ならproxyサーバを設定する
#PROXY = '127.0.0.1:8080'
#options.add_argument('--proxy-server=http://%s' % PROXY)
# ブラウザはヘッドレス（表示しない）とする
options.add_argument('--headless')
# Chromeのドライバを得る
browser = webdriver.Chrome(options=options)

# 暗黙的な待機を最大3秒行う(サーバーの負担軽減)
browser.implicitly_wait(3)
# URLを読み込む
browser.get(url)
# htmlを取得
search_results_html = browser.page_source
# 検索結果の件数からページ数を得る（検索結果は１ページあたり２０件までのため）
results_num = re.search(r'<div aria-label="page counter".*?<span class="total.*?>(.*?)<span class="unit.*?</div>',search_results_html,re.DOTALL).group(1)
print(results_num)
#pages_num = -(-(int(results_num))//20)
pages_num = 1  # 検索結果のはじめの１ページ（２０件）分のみに制限

file = open(urllib.parse.unquote(keywords) + 'articles.txt', 'a', encoding='UTF-8')

search_results = re.search(r'<div><time class="dateHeadline(.*?)<p class',search_results_html,re.DOTALL).group(1)
#print(search_results)
get_article_to_text(search_results, file)

for i in range(pages_num-1):
  url = 'https://www.nikkei.com/pressrelease/?searchKeyword='+ keywords + '&page='+ str(i+2)
  # 暗黙的な待機を最大3秒行う(サーバーの負担軽減)
  browser.implicitly_wait(3)
  # URLを読み込む
  browser.get(url)
  # htmlを取得
  search_results_html = browser.page_source

  search_results = re.search(r'<div><time class="dateHeadline(.*?)<p class',search_results_html,re.DOTALL).group(1)

  get_article_to_text(search_results, file)

# ブラウザを終了
browser.quit()
file.close()

44
https://www.nikkei.com/article/DGXZRSP690485_Q5A430C2000000/
東北大、パナソニックHDなどと共同でIPHatchの枠組みを活用したスタートアップ企業への特許権の譲渡を実施 2025年4月30日 16:59
https://www.nikkei.com/article/DGXZRSP690140_S5A420C2000000/
パナソニック、毎日の調理をサポートするAIパートナー「Bistro アシスタント」を開始 2025年4月22日 15:58
https://www.nikkei.com/article/DGXZRSP690098_S5A420C2000000/
JEITA、3月のパーソナルコンピュータ国内出荷実績を発表 2025年4月22日 10:50
https://www.nikkei.com/article/DGXZRSP689612_Q5A410C2000000/
三菱ガス化学、パナソニックと共同でCO2から製造したメタノールを原料とする環境配慮型ユリア樹脂を開発 2025年4月10日 17:19
https://www.nikkei.com/article/DGXZRSP689452_Y5A400C2000000/
パナソニックインフォメーションシステムズ、社内システムへのテックタッチ導入により業務の習得時間やサポートコストを削減 2025年4月8日 12:17
https://www.nikkei.com/article/DGXZRSP689355_U5A400C2000000/
パナソニック ハウジングソリューションズなど、「木造軸組工法向け邸別構造計算」サービスを提供開始 2025年4月4日 14:49
https://www.nikkei.com/article/DGXZRSP688921_W5A320C2000000/
パナソニックホームズ、「くらし研究室」が「家事の意識と実態に関する調査」の結果を発表 2025年3月26日 16:54
https://www.nikkei.com/article/DGXZRSP688783_V20C25A3000000/
JEITA、2月のパーソナルコンピュータ国内出荷実績を発表 2025年3月25日 11:34
https://www.n